# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [3]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [4]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [5]:
patient = pd.read_excel(path + 'audio_location.xlsx')

In [6]:
patient

,filename,위도,경도
0,audio1.mp3,37.358618,127.115036
1,audio2.mp3,36.815571,127.128844
2,audio3.mp3,37.538435,126.989828
3,audio4.mp3,35.185745,129.076541
4,audio5.mp3,36.503544,127.252941
5,1_1,37.566500,126.978000
6,1_2,35.179600,129.075600
7,2_1,35.871400,128.601400
8,2_2,37.456300,126.705200
9,3_1,35.159500,126.852600


In [7]:
data = pd.read_csv(path + '응급실정보.csv')

In [8]:
data

,기관명,주소,응급의료기관종류,대표전화,응급실전화,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
520,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",지역응급의료센터,031-467-9717,031-467-9119,37.393404,126.924477
521,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",지역응급의료센터,031-389-3000,031-389-3119,37.358645,126.947360
522,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),응급실운영신고기관,051-709-3000,051-709-3119,35.185413,129.121459
523,흑룡의원,인천광역시 옹진군 백령면 백령로 831,응급실운영신고기관,032-837-6873,032-837-3153,37.959524,124.665499


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [9]:
haversine((data['위도'][0], data['경도'][0]), (37.3586184462591, 127.115035898293), unit='km')

280.9774581542671

#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [ ]:
# 입력된 좌표와 전체 응급실과의 거리 계산
# 가장 가아운 거리 응급실 3곳을 반환


In [10]:
input = (patient['위도'][0], patient['경도'][0])

In [11]:

# 가장 가까운 거리의 응급실 3곳 반환
def recommend_hospital1(input, data):
    # 거리 계산 및 정렬
    nearest = sorted(
        data.itertuples(index=False),
        key=lambda row: haversine(input, (row.위도, row.경도), unit='km')
    )
    # 가장 가까운 top_n 응급실 반환
    return {
        row.기관명: haversine(input, (row.위도, row.경도), unit='km')
        for row in nearest[:3]
    }

In [12]:
# 결과 출력
result1 = recommend_hospital1(input, data)
print(result1)

{'분당서울대학교병원': 1.111161722006186, '대진의료재단분당제생병원': 3.29996370427956, '국군수도병원': 4.7388990676761376}


In [13]:
result1 = pd.DataFrame(result1.items(), columns=['기관명', '거리'])
result1

,기관명,거리
0,분당서울대학교병원,1.111162
1,대진의료재단분당제생병원,3.299964
2,국군수도병원,4.738899


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [14]:
# 일정범위 네모안에서 찾아보고(위도, 경도 값에 ± α), 응급실 정보에서 해당 구간을 먼저 조회한 후, 거리를 계산하여 3곳 추천하기

In [15]:
def recommend_hospital2(input, data, alpha=0.1, top_n=3):

    while True:
        # 범위 계산
        lat_min, lat_max = input[0] - alpha, input[0] + alpha
        lon_min, lon_max = input[1] - alpha, input[1] + alpha

        # 범위 내 좌표 필터링
        filtered_data = data[
            (data['위도'] >= lat_min) & (data['위도'] <= lat_max) &
            (data['경도'] >= lon_min) & (data['경도'] <= lon_max)
        ]

        # 응급실이 top_n개 이상인 경우 종료
        if len(filtered_data) >= top_n:
            break

        # α 값 증가
        alpha *= 1.5  # 증가율 조정 가능

    # 거리 계산 및 정렬
    nearest = sorted(
        filtered_data.itertuples(index=False),
        key=lambda row: haversine(input, (row.위도, row.경도), unit='km')
    )

    # 가장 가까운 top_n 응급실 반환
    return {
        row.기관명: haversine(input, (row.위도, row.경도), unit='km')
        for row in nearest[:top_n]
    }

In [16]:
# 결과 출력
result2 = recommend_hospital2(input, data)
print(result2)

{'분당서울대학교병원': 1.111161722006186, '대진의료재단분당제생병원': 3.29996370427956, '국군수도병원': 4.7388990676761376}


In [17]:
result2 = pd.DataFrame(result2.items(), columns=['기관명', '거리'])
result2

,기관명,거리
0,분당서울대학교병원,1.111162
1,대진의료재단분당제생병원,3.299964
2,국군수도병원,4.738899


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [18]:
with open(path + 'map_key.txt', 'r') as file:
    key = json.load(file)

# 딕셔너리에서 값 추출
c_id = key['c_id']  # 예시 키 이름
c_key = key['c_key']  # 예시 키 이름

#### 2) 함수 생성

In [19]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response =  requests.get(url, headers=headers, params=params)

    # 응답 상태 코드 확인
    if response.status_code == 200:  # 성공
        data = response.json()
        # 거리 추출
        dist = data['route']['trafast'][0]['summary']['distance']  # m(미터)
        return dist
    else:  # 실패
        raise Exception(f"API 호출 실패: {response.status_code}, {response.text}")

* 테스트

In [20]:
get_dist(patient['위도'][0], patient['경도'][0], data['위도'][0], data['경도'][0], c_id, c_key)

354566

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [21]:
def recommend_hospital3(input, data, c_id, c_key, alpha=0.1, top_n=3):
    while True:
        # 범위 계산
        lat_min, lat_max = input[0] - alpha, input[0] + alpha
        lon_min, lon_max = input[1] - alpha, input[1] + alpha

        # 범위 내 좌표 필터링
        filtered_data = data[
            (data['위도'] >= lat_min) & (data['위도'] <= lat_max) &
            (data['경도'] >= lon_min) & (data['경도'] <= lon_max)
        ]

        # 응급실이 top_n개 이상인 경우 종료
        if len(filtered_data) >= top_n:
            break

        # α 값 증가
        alpha *= 1.5  # 증가율 조정 가능

    # 거리 계산 및 정렬
    nearest = sorted(
        filtered_data.itertuples(index=False),
        key=lambda row: get_dist(input[0],input[1],row.위도, row.경도, c_id, c_key)
    )

    # 가장 가까운 top_n 응급실 반환
    return {
        row.기관명: get_dist(input[0],input[1],row.위도, row.경도, c_id, c_key)
        for row in nearest[:top_n]
    }

In [22]:
result3 = recommend_hospital3(input,data, c_id, c_key)
result3

{'분당서울대학교병원': 2626, '대진의료재단분당제생병원': 4707, '국군수도병원': 5973}

In [23]:
result3 = pd.DataFrame(result3.items(), columns=['기관명', '거리(m)'])
result3

,기관명,거리(m)
0,분당서울대학교병원,2626
1,대진의료재단분당제생병원,4707
2,국군수도병원,5973


In [31]:
data = pd.read_csv(path + '응급실정보.csv')
data.head()
# DataFrame 생성
emergency_df = pd.DataFrame(data)

# 응급기관 데이터를 리스트로 변환
emergency_locations = []

for _, row in emergency_df.iterrows():
    emergency_locations.append({"name": row["기관명"], "coords": (row["위도"], row["경도"])})

# 결과 출력
emergency_locations

[{'name': '(의)내경의료재단울산제일병원',
  'coords': (35.54823820112527, 129.30701143429678)},
 {'name': '(의)서일의료재단기장병원', 'coords': (35.23602946449906, 129.21649161387128)},
 {'name': '(의)성세의료재단 뉴성민병원', 'coords': (37.5089936801167, 126.669478649026)},
 {'name': '(의)영문의료재단다보스병원',
  'coords': (37.234641294534285, 127.21049868474802)},
 {'name': '(의)효심의료재단용인서울병원', 'coords': (37.240316373, 127.2144914405)},
 {'name': '(재)미리내천주성삼성직수도회천주성삼병원',
  'coords': (35.84022544517349, 128.70573339664242)},
 {'name': 'KS병원', 'coords': (35.197963178976366, 126.83322475289852)},
 {'name': '가톨릭관동대학교국제성모병원', 'coords': (37.54237265391807, 126.6836048687878)},
 {'name': '가톨릭대학교 인천성모병원', 'coords': (37.4845586499062, 126.724708130675)},
 {'name': '가톨릭대학교부천성모병원', 'coords': (37.4874477394264, 126.79243459155)},
 {'name': '가톨릭대학교성빈센트병원', 'coords': (37.27792234538125, 127.02744945028702)},
 {'name': '가톨릭대학교여의도성모병원', 'coords': (37.51827233800711, 126.93673129599132)},
 {'name': '가톨릭대학교은평성모병원', 'coords': (37.63360840972685, 126

In [28]:
def recommend_hospitals(user_location, emergency_locations, API_KEY_ID, API_KEY, max_distance=10, max_attempt_distance=50):
    import requests
    from math import radians, sin, cos, sqrt, atan2
    # Haversine 함수
    def haversine(coord1, coord2):
        R = 6371.0  # 지구 반지름 (km)
        lat1, lon1 = radians(coord1[0]), radians(coord1[1])
        lat2, lon2 = radians(coord2[0]), radians(coord2[1])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c

    # Directions API 호출 함수
    def get_travel_time_with_fallback(start_coords, end_coords):
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        options = ["trafast", "traoptimal"]  # 사용할 옵션 순서
        headers = {
            "X-NCP-APIGW-API-KEY-ID": API_KEY_ID,
            "X-NCP-APIGW-API-KEY": API_KEY
        }

        for option in options:
            params = {
                "start": f"{start_coords[1]},{start_coords[0]}",  # 경도, 위도 순서
                "goal": f"{end_coords[1]},{end_coords[0]}",
                "option": option
            }

            try:
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()

                # 'route' 데이터 확인 및 처리
                if "route" in data and option in data["route"]:
                    summary = data["route"][option][0]["summary"]
                    duration_ms = summary["duration"]  # 소요 시간 (밀리초)
                    distance_m = summary["distance"]  # 도로 거리 (미터)
                    return duration_ms / 1000, distance_m / 1000  # 초 단위, km 단위 반환
            except requests.exceptions.RequestException as e:
                print(f"{option} 옵션 실패: {e}")
            except KeyError as e:
                print(f"API 응답에서 예상치 못한 데이터 구조: {e}")

        # 모든 옵션 실패 시
        return float("inf"), float("inf")

    # 거리 기준 확장하며 병원 검색
    current_distance = max_distance
    while current_distance <= max_attempt_distance:
        filtered_hospitals = [
            loc for loc in emergency_locations
            if haversine(user_location, loc["coords"]) <= current_distance
        ]

        if filtered_hospitals:
            results = []
            for hospital in filtered_hospitals:
                travel_time, road_distance = get_travel_time_with_fallback(user_location, hospital["coords"])
                if not (travel_time is None or travel_time == float("inf")):  # 유효한 값만 추가
                    results.append({
                        "name": hospital["name"],
                        "road_distance": road_distance,
                        "travel_time": travel_time
                    })

            if results:  # 유효한 결과가 있는 경우
                top_3_hospitals = sorted(results, key=lambda x: x["travel_time"])[:3]

                print("추천 병원:")
                for hospital in top_3_hospitals:
                    duration_in_seconds = hospital["travel_time"]  # 초 단위 값
                    hours = int(duration_in_seconds // 3600)
                    minutes = int((duration_in_seconds % 3600) // 60)
                    print(
                        f"- {hospital['name']}, 도로 거리: {hospital['road_distance']:.2f}km, "
                        f"예상 소요 시간: {hours}시간 {minutes}분"
                    )
                return  # 추천 병원 출력 후 종료
        current_distance += 10

    # 최대 거리까지 검색했지만 병원 없음
    print(f"{max_attempt_distance}km 내 유효한 병원을 찾을 수 없습니다.")


In [32]:
user_location = (37.5034138, 126.7660309)  # 사용자 좌표 예시
recommend_hospitals(user_location, emergency_locations, c_id, c_key)

추천 병원:
- 순천향대학교부속부천병원, 도로 거리: 1.28km, 예상 소요 시간: 0시간 5분
- 의료법인대인의료재단다니엘종합병원, 도로 거리: 1.58km, 예상 소요 시간: 0시간 7분
- 뉴대성병원, 도로 거리: 2.72km, 예상 소요 시간: 0시간 11분


## **Mission Complete!**

수고 많았습니다!